修改下面的数据库配置来爬

In [44]:
import requests
from bs4 import BeautifulSoup
import re,json
import pymysql


# 数据库连接配置
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Mo01iHt++',
    'database': 'lenovoshop',
    'charset': 'utf8'
}



# 目标网页的URL
url = "https://item.lenovo.com.cn/"

# 发送HTTP请求获取页面内容
response = requests.get(url)
response.encoding = 'utf-8'  # 确保使用正确的编码格式

# 检查请求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 查找所有符合条件的<div>标签
    divs = soup.find_all('div', attrs={'type': '2'})
    
    # 过滤order属性在4到15之间的<div>
    filtered_divs = [div for div in divs if 'order' in div.attrs and 4 <= int(div['order']) <= 15]
    
     # 初始化字典存储数据
    href_dict = {}
   
    # 遍历符合条件的<div>，并查找<ul class="product-list-ul">中的所有<li>
    for div in filtered_divs:
        # 查找<span class="level-name-left">的内容作为类别名
        span = div.find('span', class_='level-name-left')
        if span:
            category_name = span.text.strip()
            print(category_name)
            if category_name == "ThinkPad 电脑":
                continue
            ul = div.find('ul', class_='product-list-ul')
            if ul:
                li_tags = ul.find_all('li')
                for li in li_tags:
                    
                    # 获取图片地址 和商品ID
                    img_tag = li.find('img')
                    img_url = img_tag['src'] if img_tag else None

                    id_tag = li.find('span',class_="price_box_price")
                    product_id =  id_tag["gcode"] if id_tag else None
                    
                    # 获取商品名称
                    name_tag = li.find('a', class_='product-des-title')
                    product_name = name_tag.text.strip() if name_tag else None
                    
                    # 获取商品简介
                    desc_tag = li.find('a', class_='product-des-subtitle')
                    description = desc_tag.text.strip() if desc_tag else None

                    #获取商品价格
                    id_tag = li.find('span',class_="price_box_price")
                    try:
                        # 尝试将 id_tag.text 转换为浮点数并乘以 100
                        price = int(float(id_tag.text) * 100)
                    except (ValueError, TypeError):
                        # 如果转换失败，或者 id_tag 为 None，则将 price 设置为 0
                        price = 114514
                    
                   
                    
                    # 打印或存储结果
                    # print(f"Image URL: {"https:"+img_url}")
                    # print(f"Product Name: {product_name}")
                    # print(f"Description: {description}")
                    # print(f"Product ID: {product_id}")
                
                    # 如果类别名不在字典中，初始化一个空列表
                    if category_name not in href_dict:
                        href_dict[category_name] = []

                    href_dict[category_name].append(["https:"+img_url,product_name,description,product_id,price])
                    
                    
                   

else:
    print("请求失败，状态码:", response.status_code)


Lenovo 电脑
Lenovo 台式机
ThinkPad 电脑
手机&配件
平板电脑
选件
服务/配件
智能
显示器
IP周边


In [45]:
print(href_dict)

{'Lenovo 电脑': [['https://p1.lefile.cn/product/adminweb/2024/03/07/uzkSGc4tdk7oPSB8NOtflkckW-3479.jpg', '小新Pro14 AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/鸽子灰', '1039020', 749900], ['https://p1.lefile.cn/product/adminweb/2024/03/25/oct7JgNqsFM1ZEp1XEOCJObQt-8655.jpg', '小新Pro16 AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/鸽子灰', '1039019', 769900], ['https://p1.lefile.cn/product/adminweb/2024/04/18/kXmUDZtgzhE6pUIEwJEi4i4O9-8223.jpg', '小新Pro16 AI元启', 'R7 8845H/32G/1T SSD/集成显卡/鸽子灰', '1037529', 639900], ['https://p3.lefile.cn/product/adminweb/2024/08/16/mbjo7NiPoQ8k0tuJF7ZMi3Xmz-3777.jpg', 'YOGA Pro 14s AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/信风灰', '1039021', 899900], ['https://p2.lefile.cn/product/adminweb/2024/08/19/wmu5yWOFOuwgq0fl8ojIaWsCn-4047.jpg', '拯救者Y9000P《黑神话：悟空》联名定制版', 'i9/32G/1T SSD/RTX 4070', '1039042', 114514], ['https://p1.lefile.cn/product/adminweb/2024/08/19/WylapNCTrteNH2B0bEulhOX5z-6933.jpg', '拯救者Y9000P《黑神话：悟空》联名定制版', 'i9/32G/1T SSD/RTX 4060', '1039043', 114514], ['https://p3.lefile.cn/fes/cms/2024/

In [46]:


# 初始化字典存储爬取结果
results = {}

# 遍历每个类别和对应的链接列表
for category, urls in href_dict.items():
    results[category] = []
    
    for url in urls:
        print(f"Processing {url} in category '{category}'...")

        pic = url[0]
        short_name = url[1]
        short_description = url[2]
        product_code = url[3]
        price = url[4]
        
        
        jsonp_url = f"https://open.lenovo.com.cn/goods/category/get?jsonpCallback=jQueryJSONP_category_get&code={product_code}&version=1.0&_={int(1e12)}"

        # 发送JSONP请求
        jsonp_response = requests.get(jsonp_url)
        
        if jsonp_response.status_code == 200:
            # 使用正则表达式提取JSON部分
            json_data = re.search(r'jQueryJSONP_category_get\((.*)\);', jsonp_response.text).group(1)
            
            # 解析JSON数据
            data = json.loads(json_data)
            
            if data["status"] == 0:
                # 获取规格数据并添加到results字典中
                specs = data["result"]
                results[category].append({
                    'pic': pic,
                    'short_name':short_name,
                    'short_description':short_description,
                    'product_code':product_code,
                    "price":price,
                    'specs': specs
                    # TODO：全名，全简介，价格
                })
            else:
                print(f"Error: {data['message']}")
        else:
            print(f"JSONP请求失败，状态码: {jsonp_response.status_code}")
        # else:
        #     print(f"请求失败，状态码: {response.status_code}")

# 打印结果
for category, products in results.items():
    print(f"Category: {category}")
    for product in products:
        print(f"  URL: {product['product_code']}_{product['short_name']}")
        for product_code, spec_list in product['specs'].items():
            print(f"Product Code: {product_code}")
            for i in spec_list:
                for key, value in i.items():
                    print(f"    {key}: {value}")


Processing ['https://p1.lefile.cn/product/adminweb/2024/03/07/uzkSGc4tdk7oPSB8NOtflkckW-3479.jpg', '小新Pro14 AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/鸽子灰', '1039020', 749900] in category 'Lenovo 电脑'...
Processing ['https://p1.lefile.cn/product/adminweb/2024/03/25/oct7JgNqsFM1ZEp1XEOCJObQt-8655.jpg', '小新Pro16 AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/鸽子灰', '1039019', 769900] in category 'Lenovo 电脑'...
Processing ['https://p1.lefile.cn/product/adminweb/2024/04/18/kXmUDZtgzhE6pUIEwJEi4i4O9-8223.jpg', '小新Pro16 AI元启', 'R7 8845H/32G/1T SSD/集成显卡/鸽子灰', '1037529', 639900] in category 'Lenovo 电脑'...
Processing ['https://p3.lefile.cn/product/adminweb/2024/08/16/mbjo7NiPoQ8k0tuJF7ZMi3Xmz-3777.jpg', 'YOGA Pro 14s AI元启', 'Ultra 9/32GB/1T SSD/集成显卡/信风灰', '1039021', 899900] in category 'Lenovo 电脑'...
Processing ['https://p2.lefile.cn/product/adminweb/2024/08/19/wmu5yWOFOuwgq0fl8ojIaWsCn-4047.jpg', '拯救者Y9000P《黑神话：悟空》联名定制版', 'i9/32G/1T SSD/RTX 4070', '1039042', 114514] in category 'Lenovo 电脑'...
Processing ['https://p1.le

In [47]:

print(results)

{'Lenovo 电脑': [{'pic': 'https://p1.lefile.cn/product/adminweb/2024/03/07/uzkSGc4tdk7oPSB8NOtflkckW-3479.jpg', 'short_name': '小新Pro14 AI元启', 'short_description': 'Ultra 9/32GB/1T SSD/集成显卡/鸽子灰', 'product_code': '1039020', 'price': 749900, 'specs': {'1039019': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'AI元启/Ultra 9/Pro 16'}, {'存储': '32G/1T SSD'}, {'显卡': '集成显卡'}], '1039020': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'AI元启/Ultra 9/Pro 14'}, {'存储': '32G/1T SSD'}, {'显卡': '集成显卡'}], '1034422': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'Ultra 5/Pro 16'}, {'存储': '16G/1T SSD'}, {'显卡': '集成显卡'}], '1034913': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'Ultra 5/Pro 14'}, {'存储': '32G/1T SSD'}, {'显卡': '集成显卡/OLED'}], '1036085': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'Ultra 9/Pro 16'}, {'存储': '32G/1T SSD'}, {'显卡': '集成显卡'}], '1035698': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'Ultra 5/Pro 14'}, {'存储': '16G/1T SSD'}, {'显卡': '集成显卡/LCD'}], '1037823': [{'操作系统': 'Windows 11 家庭中文版'}, {'处理器': 'Ultra 5/Pro 14'}, {'存储': '32G/1T SS

In [48]:

def is_config_exists(cursor, config_id):
    query = "SELECT COUNT(*) FROM configs WHERE product_code = %s"
    cursor.execute(query, (config_id,))
    count = cursor.fetchone()[0]
    return count > 0

def category_exists(cursor, category_name):
    sql = "SELECT id FROM categories WHERE name = %s"
    cursor.execute(sql, (category_name,))
    return cursor.fetchone()

def insert_category(cursor, category_name):
    if category_exists(cursor, category_name):
        return category_exists(cursor, category_name)[0]  # Return the existing category id
    sql = "INSERT INTO categories (name) VALUES (%s)"
    cursor.execute(sql, (category_name,))
    return cursor.lastrowid

def insert_config(cursor,product_id, config_name,brief, price, value, product_code):
    sql = "INSERT INTO configs (product_id,name,brief, price, value, product_code) VALUES (%s, %s, %s,%s,%s, %s)"
    
    cursor.execute(sql, (product_id,config_name, brief,price, json.dumps(value), product_code))
    return cursor.lastrowid

def insert_product(cursor, product_id, name,price,brief, pic_url, category_id):
    sql = "INSERT INTO products (product_id, name, price,brief,pic_url, category_id) VALUES (%s, %s, %s, %s,%s,%s)"
    cursor.execute(sql, (product_id, name, price,brief,pic_url, category_id))
    return cursor.lastrowid

def main():
    connection = pymysql.connect(**db_config)
    try:
        with connection.cursor() as cursor:
            for category_name, products in results.items():
                category_id = insert_category(cursor, category_name)
                
                for product in products:
                    pic = product['pic']
                    product_id = product['product_code']
                    short_name = product['short_name']
                    short_description = product['short_description']
                    price = product['price']

                    new_product_id = insert_product(cursor, product_id, short_name, price,short_description,pic, category_id)
                    
                    for config_id, specs in product['specs'].items():
                        

                        url = 'https://papi.lenovo.com.cn/batch/get'
                        params = [{"uri": "/batch/openapi/goods/detail/mget/B00001", "param": {"code": config_id}}]
                        callback = 'jQueryJSONP_batch_get'

                        jsonp_response = requests.get(url, params={'params': json.dumps(params), 'callback': callback})

                        
                        if jsonp_response.status_code == 200:
                            # 使用正则表达式提取JSON部分
                            json_data = re.search(r'jQueryJSONP_batch_get\((.*)\)', jsonp_response.text).group(1)
                            
                            # 解析JSON数据
                            data = json.loads(json_data)
                            #print(data)
                            if data["status"] == 200:

                                item = data["data"][0]['result'][config_id]
                                print(item)
                                config_name = item['detail']['name']
                                description = item['detail']['brief']
                                price = int(float(item['price']['basePrice']) * 100)  # Set the price as needed
                                
                                # Convert specs to JSON string
                                value = [{spec_key: spec_value} for spec in specs for spec_key, spec_value in spec.items()]
                                if not is_config_exists(cursor, config_id):
                                    insert_config(cursor, new_product_id,config_name, description,price, value, config_id)
                            
                        

            connection.commit()
    finally:
        connection.close()

if __name__ == "__main__":
    main()



{'price': {'P_C': {'C_1': {'P_0': {'DV': {'V': 7699}}}, 'C_5': {'P_0': {'DV': {'V': 7699}}}, 'C_4': {'P_0': {'DV': {'V': 7699}}}, 'C_3': {'P_0': {'DV': {'V': 7699}}}, 'C_2': {'P_0': {'DV': {'V': 7699}}}}, 'basePrice': 7699}, 'activity': {}, 'detail': {'marketableName': '上架', 'url': {'wap': 'https://mitem.lenovo.com.cn/product/1039019.html', 'app': 'https://mitem.lenovo.com.cn/android/product/1039019.html', 'pc': 'https://item.lenovo.com.cn/product/1039019.html'}, 'adPicture': '', 'code': 1039019, 'thumbnail': {'path': '//p3.lefile.cn/product/adminweb/2024/08/16/Hzy6pwhqKWgwjQ5xQGHLvbfeO-3512.jpg', 'id': '723a21bd-feb4-4b9b-8870-9e2073231851'}, 'isShowPrice': True, 'isIndependentStock': False, 'mallType': 1, 'faName': '联想 ( 上海 ) 电子科技有限公司', 'isPhysical': '1', 'materialNumber': '83D4006NCD', 'isService': 0, 'serviceTypeName': '普通商品', 'productGroupId': 'c324f432-f156-4796-9e23-11edb34e5630', 'isCountSalesVolume': True, 'buOwner': '70085', 'isUpgradeGoods': False, 'isHouda': False, 'name': 

In [49]:
print(jsonp_response.text)

jQueryJSONP_category_get({"result":{"1035066":[{"型号":"双肩包"},{"颜色":"YOGA*高迪联名双肩包\t"}],"1035067":[{"型号":"托特包"},{"颜色":"YOGA*高迪联名托特包\t"}]},"status":0,"message":"查询成功"});

